In [ ]:
import os
from datetime import timedelta
from unittest import TestCase

import inflect
import pytest
import subprocess

from ska_dlm import CONFIG, data_item, dlm_ingest, dlm_migration, dlm_request, dlm_storage
from ska_dlm.dlm_db.db_access import DB
from ska_dlm.dlm_storage.main import persist_new_data_items
from ska_dlm.exceptions import InvalidQueryParameters, ValueAlreadyInDB

Set-up services:

In [ ]:
subprocess.run(['make', 'python-pre-test'], check=True)
!pgrep postgrest

In [ ]:
def _clear_database():
    DB.delete(CONFIG.DLM.dlm_table)
    DB.delete(CONFIG.DLM.storage_config_table)
    DB.delete(CONFIG.DLM.storage_table)
    DB.delete(CONFIG.DLM.location_table)

Define storage location and configure r-clone:

In [ ]:
# @pytest.fixture(scope="function", autouse=True)
# def setup_and_teardown(self):
"""Initialze the tests."""
_clear_database()

# we need a location to register the storage
location_id = dlm_storage.init_location("MyOwnStorage", "Server")
uuid = dlm_storage.init_storage(
    storage_name="MyDisk",
    location_id=location_id,
    storage_type="disk",
    storage_interface="posix",
    storage_capacity=100000000,
)
config = '{"name":"MyDisk","type":"local", "parameters":{}}'
dlm_storage.create_storage_config(uuid, config=config)
# configure rclone:
dlm_storage.rclone_config(config)
    # yield
    # _clear_database()

Intialize a new data_item:

In [ ]:
# test_init(self):
"""Test data_item init."""
engine = inflect.engine()
success = True
for i in range(1, 51, 1):
    ordinal = engine.number_to_words(engine.ordinal(i))
    uid = dlm_ingest.init_data_item(f"this/is/the/{ordinal}/test/item")
    if uid is None:
        success = False
assert success
print("True")

Ingest a data item:

In [ ]:
# def test_ingest_data_item(self):
"""Test the ingest_data_item function."""
uid = dlm_ingest.ingest_data_item("/my/ingest/test/item", "/LICENSE", "MyDisk")
if len(uid) == 36:
    print('Pass')
else:
    print('Fail')
# assert len(uid) == 36

Open the GUI in the browser:

In [ ]:
# change to the PostGUI repository directory:
!cd /Users/00110564/ska/PostGUI \
    && npm install 2>/dev/null \
    && export NODE_OPTIONS=--openssl-legacy-provider \
    && npm start

# This process runs in the background - kill manually.

In [ ]:
_clear_database()

Tear down services:

In [ ]:
!make python-post-test